# Webscraping do site Alura Motors

<p>Passo a passo. Ao final, código completo organizado</p>

## Importando bibliotecas e criando objeto do beautifulsoup

In [1]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://alura-site-scraping.herokuapp.com/index.php'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

req = Request(url, headers = headers)

try:
    req = Request(url, headers = headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
except HTTPError as e:
    print(e.status, e.reason)
except URLError as e:
    print(e.reason)

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>

<html lang="pt-br">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Alura Motors</title>
<style>
		/*Regra para a animacao*/
		@keyframes spin {
			0% { transform: rotate(0deg); }
			100% { transform: rotate(360deg); }
		}
		/*Mudando o tamanho do icone de resposta*/
		div.glyphicon {
			color:#6B8E23;
			font-size: 38px;
		}
		/*Classe que mostra a animacao 'spin'*/
		.loader {
			border: 16px solid #f3f3f3;
			border-radius: 50%;
			border-top: 16px solid #3498db;
			width: 80px;
			height: 80px;
			-webkit-animation: spin 2s linear infinite;
			animation: spin 2s linear infinite;
		}
	</style>
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" rel="stylesheet"/>
<link href="css/styles.css" media="all" rel="stylesheet"/>
<script src="https://code.jquery

## Passo a passo para encontrar informações nos anúncios

In [4]:
cards = []
card = {}

In [5]:
anuncio = soup.find('div', {'class': 'well card'})

In [6]:
anuncio

<div class="well card">
<div class="col-md-3 image-card">
<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>
</div>
<div class="col-md-6 body-card">
<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>
<p class="txt-category badge badge-secondary inline">USADO</p>
<p class="txt-motor">Motor 1.8 16v</p>
<p class="txt-description">Ano 1993 - 55.286 km</p>
<ul class="lst-items">
<li class="txt-items">► 4 X 4</li>
<li class="txt-items">► Câmera de estacionamento</li>
<li class="txt-items">► Controle de tração</li>
<li class="txt-items">► Sensor de estacionamento</li>
<li class="txt-items">...</li>
</ul>
<p class="txt-location">Belo Horizonte - MG</p>
</div>
<div class="col-md-3 value-card">
<div class="value">
<p class="txt-value">R$ 338.000</p>
</div>
</div>
</div>

In [7]:
valor_anuncio = anuncio.find('div', {'class': 'value-card'}).get_text()

In [8]:
valor_anuncio = valor_anuncio.split()

In [9]:
valor_anuncio = " ".join(valor_anuncio)

In [10]:
valor_anuncio

'R$ 338.000'

In [11]:
card['value'] = valor_anuncio

In [12]:
descricao_anuncio = anuncio.find('div', {'class': 'body-card'}).find_all('p')

In [13]:
descricao_anuncio

[<p class="txt-name inline">LAMBORGHINI AVENTADOR</p>,
 <p class="txt-category badge badge-secondary inline">USADO</p>,
 <p class="txt-motor">Motor 1.8 16v</p>,
 <p class="txt-description">Ano 1993 - 55.286 km</p>,
 <p class="txt-location">Belo Horizonte - MG</p>]

In [14]:
for descricao in descricao_anuncio:
    print(descricao.get('class')[0].split('-')[-1], descricao.get_text())
    chave = descricao.get('class')[0].split('-')[-1]
    valor = descricao.get_text()
    card[chave] = valor

name LAMBORGHINI AVENTADOR
category USADO
motor Motor 1.8 16v
description Ano 1993 - 55.286 km
location Belo Horizonte - MG


In [15]:
card

{'value': 'R$ 338.000',
 'name': 'LAMBORGHINI AVENTADOR',
 'category': 'USADO',
 'motor': 'Motor 1.8 16v',
 'description': 'Ano 1993 - 55.286 km',
 'location': 'Belo Horizonte - MG'}

In [16]:
acessorios_anuncio = anuncio.find('div', {'class': 'body-card'}).ul.findAll('li')
acessorios_anuncio.pop()
acessorios = []

In [17]:
for item in acessorios_anuncio:
    acessorios.append(item.getText().replace('► ', ''))
card['items'] = acessorios

In [18]:
card

{'value': 'R$ 338.000',
 'name': 'LAMBORGHINI AVENTADOR',
 'category': 'USADO',
 'motor': 'Motor 1.8 16v',
 'description': 'Ano 1993 - 55.286 km',
 'location': 'Belo Horizonte - MG',
 'items': ['4 X 4',
  'Câmera de estacionamento',
  'Controle de tração',
  'Sensor de estacionamento']}

In [19]:
image = anuncio.find('div', {'class': 'image-card'}).img
image

<img alt="Foto" height="155" src="https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg" width="220"/>

In [20]:
image.get('src')

'https://caelum-online-public.s3.amazonaws.com/1381-scraping/01/img-cars/lamborghini-aventador/lamborghini-aventador-2932196__340.jpg'

In [21]:
from IPython.core.display import display, HTML

display(HTML(str(anuncio.find('div', {'class': 'image-card'}).img)))

# Automatizando a rotina de scraping

In [22]:
#pegando todos os cards de uma vez
anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")

In [23]:
len(anuncios)

10

In [24]:
for anuncio in anuncios:
    card = {}

    # Valor
    card['value'] = anuncio.find('p', {'class': 'txt-value'}).getText()

    # Informações
    infos = anuncio.find('div', {'class': 'body-card'}).findAll('p')
    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text()

    # Acessórios
    items = anuncio.find('div', {'class': 'body-card'}).ul.findAll('li')
    items.pop()
    acessorios = []
    for item in items:
        acessorios.append(item.get_text().replace('► ', ''))
    card['items'] = acessorios

    # Adicionando resultado a lista cards
    cards.append(card)

    # Imagens
    image = anuncio.find('div', {'class': 'image-card'}).img
    urlretrieve(image.get('src'), 'img/' + image.get('src').split('/')[-1])

## ... O que fazer quando a lista está separada por índices na página?

<p> Há situações em que será necessário interagir com o site, para coletar informações que estão paginadas, ou seja, divididas por índices </p>

In [25]:
paginas = soup.find('span', class_='info-pages').get_text().split()[-1]
paginas

'25'

# Webscraping da página inteira

In [26]:
# Importando bibliotecas
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

# Declarando variável cards
cards = []

# Obtendo o HTML...
response = urlopen('https://alura-site-scraping.herokuapp.com/index.php')
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')

# ... para obter o total de páginas
pages = int(soup.find('span', class_="info-pages").get_text().split()[-1])

# Iterando por todas as páginas do site
for i in range(pages):
    ## Obtendo o HTML
    response = urlopen('https://alura-site-scraping.herokuapp.com/index.php?page=' + str(i  + 1))
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    
    # Coletando os cards
    anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card")
    
    for anuncio in anuncios:
        card = {}

        # Valor
        card['value'] = anuncio.find('p', {'class': 'txt-value'}).getText()

        # Informações
        infos = anuncio.find('div', {'class': 'body-card'}).findAll('p')
        for info in infos:
            card[info.get('class')[0].split('-')[-1]] = info.get_text()

        # Acessórios
        items = anuncio.find('div', {'class': 'body-card'}).ul.findAll('li')
        items.pop()
        acessorios = []
        for item in items:
            acessorios.append(item.get_text().replace('► ', ''))
        card['items'] = acessorios

        # Adicionando resultado a lista cards
        cards.append(card)

        # Imagens
        image = anuncio.find('div', {'class': 'image-card'}).img
        urlretrieve(image.get('src'), 'img/' + image.get('src').split('/')[-1])

#Criando um dataframe
dataset = pd.DataFrame(cards)
dataset.to_csv('dataframe/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [27]:
dataframe = pd.read_csv('dataframe/dataset.csv', sep=';')

In [28]:
dataframe

,value,name,category,motor,description,location,items,opportunity
0,R$ 338.000,LAMBORGHINI AVENTADOR,USADO,Motor 1.8 16v,Ano 1993 - 55.286 km,Belo Horizonte - MG,"['4 X 4', 'Câmera de estacionamento', 'Control...",NaN
1,R$ 346.000,BMW M2,USADO,Motor 3.0 32v,Ano 2018 - 83.447 km,Belo Horizonte - MG,"['Câmera de estacionamento', 'Controle de esta...",NaN
2,R$ 480.000,ALFA,USADO,Motor 1.8 16v,Ano 2004 - 19.722 km,Rio de Janeiro - RJ,"['Central multimídia', 'Bancos de couro', 'Rod...",NaN
3,R$ 133.000,PUECH,USADO,Motor Diesel V8,Ano 1992 - 34.335 km,São Paulo - SP,"['Bancos de couro', 'Freios ABS', 'Rodas de li...",NaN
4,R$ 175.000,LAMBORGHINI MURCIELAGO,USADO,Motor 1.0 8v,Ano 1991 - 464 km,Belo Horizonte - MG,"['Central multimídia', 'Teto panorâmico', 'Sen...",NaN
...,...,...,...,...,...,...,...,...
241,R$ 489.000,SUV REAR TIRE,USADO,Motor 3.0 32v,Ano 1998 - 74.292 km,São Paulo - SP,"['Câmera de estacionamento', 'Rodas de liga', ...",NaN
242,R$ 427.000,ANTIQUE,NOVO,Motor 2.0 16v,Ano 2019 - 0 km,Belo Horizonte - MG,"['Bancos de couro', 'Freios ABS', 'Sensor de e...",NaN
243,R$ 203.000,SPORT,USADO,Motor 2.0 16v,Ano 2001 - 102.776 km,Belo Horizonte - MG,"['Sensor crepuscular', 'Sensor de chuva', 'Vid...",NaN
244,R$ 474.000,IMPERIAL,USADO,Motor 1.8 16v,Ano 2011 - 101.787 km,Belo Horizonte - MG,"['Painel digital', 'Travas elétricas', 'Sensor...",OPORTUNIDADE
